In [3]:
# creating a large table
import sqlite3 as sql

conn = sql.connect('test.db')
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS test (id INTEGER PRIMARY KEY, name TEXT, age INTEGER, address TEXT, salary REAL)')
# 100000000 4.7G (4m 41s)
for i in range(100000):
	c.execute('INSERT INTO test VALUES (?, ?, ?, ?, ?)', (i, 'name'+str(i), i, 'address'+str(i), i*1000))
conn.commit()
conn.close()


In [10]:
from bs4 import BeautifulSoup
htmlFile = open("course _directory.html","r",encoding="utf8")
html = htmlFile.read()


In [27]:
soup = BeautifulSoup(html, 'html.parser')
soup.prettify()
tds = soup.find_all('td')
for td in tds:
	if td.text.strip() !="\n":
		print(td.get_text(separator=","))




COM101,[Undergraduate] 
Effective Reading and Comprehension Skills
3
Ananya Desai,Chirag Trivedi,Gatha Joshipura,Preeti Maneck,Purabi Bhattacharya,Rohit Majumdar,Siddharth Adesara
Monsoon
PREQ_OR,COM 100 Elements of Academic Reading and Writing,[ COM100 or should have performed well in the English Competency Test ]

COM101 is the intermediate level of the university’s communication courses designed to promote linguistic and academic development. This is a reading and comprehension course to train students to develop general competence and advanced analytical strategies in reading. This course prepares students to communicate in English at the level required for success in their core courses and beyond graduation.,
,View/Print Outline, 
None

,Section 1  ,Tue [08:00 to 09:30]
														[29-08-2022 to 30-11-2022],Thu [08:00 to 09:30]
														[29-08-2022 to 30-11-2022],Section 4  ,Tue [08:00 to 09:30]
														[29-08-2022 to 16-12-2022],Thu [08:00 to 09:30]
														[2